In [18]:
import generator_object, write_run_plot, serial_sim_tools
import numpy as np

from importlib import reload
for obj in [generator_object, write_run_plot, serial_sim_tools]:
    reload(obj)

Continuuing for John sims with rho = 6.9, N=50, T = 592 Kelvin, Z = 11, Inverse brehmmstralung, Non lte

Varying: rho, T, P, internal energy, charge state, meanfree path


In [8]:
def sim(var1, var2, var3):

    gen = generator_object.User_input()
    n_atom = 50
    rho, T_ev = var1, var2
    z = var3
    N0, N1 = 1, 501 #We are operating in 1d wih 40 nodes
    Rmin, Rmax = 0, 0.01
    p2 = 1.e-9


    gen.materials_atom(index = 1, quantum_n_max = 3, element= "sn")  
    gen.materials_region(nodes = [N0, N1], elec_temp = T_ev, qstart = True) 

    gen.materials_region_material(rho, n_atom, z, z**2) #

    gen.geometry(type = 'slab')
    gen.geometry_nodes(coordinate = "r", scaling_type  = "geom", nodes = [N0, N1], nodes_range = [Rmin, Rmax], drmin = 1.e-6, slope = -1)


    gen.sources_laser(index = 1, laser_wavelength = 1, option_1= 'value', option_2= 'history', id_value = 1, multiplier=1,)
    gen.sources_lasray(entrance_position =  10, entrance_direction_mu = 1, entrance_direction_phi = 0, fractional_power= 1, res_frac=.5)
    
    gen.sources_history(id = 1, value_multiplier = 1e18, time_multiplier = 1., pulse_type='gaussian', p1 = 5*p2, p2 = p2)

    gen.popular_switches(timestep_between_snapshot = 100, 
                        temparture_calc_heating_rates = ['temp calc = time dependant', 'heating rates = electronic'], 
                        raytrace = True, 
                        include_degeneracy = 'no degeneracy',
                        continuum_transfer_evolves_temp = True)

        
    gen.other_switches(resonant_absorption_fraction = 'constant value for each ray from lasray', 
                        subcycle_maximum = 1000,
                        do_kinetics_zone_centered = True, 
                        population_calculation = 'time dependent diffusion', 
                        control_calc_thermal_conduct = 'include thermal conduction')
    
    gen.controls(t_start = 0, t_end = 2.e-8, restart = True)

    gen.parameters(time_between_snapshots = 1e-9, initial_timestep = 1.e-14)
    
    return gen




In [9]:
var = np.array([.5, .75, 1, 1.33, 2.])

Tvals = [2*592./11600]
rhovals = 6.9*var
z_vals = [12]

combi = serial_sim_tools.combinatorics([Tvals, rhovals, z_vals])
#for comb in combi:
#    print(comb)
sims = []



for entry in combi:
    base_name = 'johnrun1'
    [var1,var2,var3] = entry
    gen = sim(var1, var2, var3)
    name = f'{base_name}_{var1}_{var2}_{var3}'
    sims.append(name)
    write_run_plot.write(name=name, longprint = False, plot_duplicates=False, object=gen)
    write_run_plot.run(name=name, longprint = False, plot_duplicates=False, object=gen)
    #write_run_plot.plot(name=name, longprint = False, plot_duplicates=False, object=gen)
serial_sim_tools.plot_all('sims_multiplot', sims)

# i'm seeing 2 minutes for 2 options, so 5*5*3 = 75 minutes maybe a bit more 


writing to /home/brewster/Desktop/cretin_package-master/Personal_experiments/johnrun1_0.10206896551724139_3.45_12/johnrun1_0.10206896551724139_3.45_12.gen
running cretin with johnrun1_0.10206896551724139_3.45_12

writing to /home/brewster/Desktop/cretin_package-master/Personal_experiments/johnrun1_0.10206896551724139_5.175000000000001_12/johnrun1_0.10206896551724139_5.175000000000001_12.gen
running cretin with johnrun1_0.10206896551724139_5.175000000000001_12

writing to /home/brewster/Desktop/cretin_package-master/Personal_experiments/johnrun1_0.10206896551724139_6.9_12/johnrun1_0.10206896551724139_6.9_12.gen
running cretin with johnrun1_0.10206896551724139_6.9_12

writing to /home/brewster/Desktop/cretin_package-master/Personal_experiments/johnrun1_0.10206896551724139_9.177000000000001_12/johnrun1_0.10206896551724139_9.177000000000001_12.gen
running cretin with johnrun1_0.10206896551724139_9.177000000000001_12

writing to /home/brewster/Desktop/cretin_package-master/Personal_experim

 

on mean free paths:
Rosseland mean absorption coefficient, including scattering, at node ir (1/cm)
Planck mean absorption coefficient, not including contributions of photoexcitations, at node ir (1/cm)

on internal energy:
Internal energy of element iz at node ir with no continuum lowering (erg/cm3).


In [11]:
# Looking for temperature, charge state, laser absorption pe unit distance, radiative heat. & other results from theoretical predictions



def sim2(var2):

    gen = generator_object.User_input()
    n_atom = 50
    rho, T_ev = 15, var2
    z = 12
    N0, N1 = 1, 501 #We are operating in 1d wih 40 nodes
    Rmin, Rmax = 0, 0.01
    p2 = 1.e-9


    gen.materials_atom(index = 1, quantum_n_max = 3, element= "sn")  
    gen.materials_region(nodes = [N0, N1], elec_temp = T_ev, qstart = True) 

    gen.materials_region_material(rho, n_atom, z, z**2) #

    gen.geometry(type = 'slab')
    gen.geometry_nodes(coordinate = "r", scaling_type  = "geom", nodes = [N0, N1], nodes_range = [Rmin, Rmax], drmin = 1.e-6, slope = -1)


    gen.sources_laser(index = 1, laser_wavelength = 1, option_1= 'value', option_2= 'history', id_value = 1, multiplier=1,)
    gen.sources_lasray(entrance_position =  10, entrance_direction_mu = 1, entrance_direction_phi = 0, fractional_power= 1, res_frac=.5)
    
    gen.sources_history(id = 1, value_multiplier = 1e18, time_multiplier = 1., pulse_type='gaussian', p1 = 5*p2, p2 = p2)

    gen.popular_switches(timestep_between_snapshot = 100, 
                        temparture_calc_heating_rates = ['temp calc = time dependant', 'heating rates = electronic'], 
                        raytrace = True, 
                        include_degeneracy = 'no degeneracy',
                        continuum_transfer_evolves_temp = True)

        
    gen.other_switches(resonant_absorption_fraction = 'constant value for each ray from lasray', 
                        subcycle_maximum = 1000,
                        do_kinetics_zone_centered = True, 
                        population_calculation = 'time dependent diffusion', 
                        control_calc_thermal_conduct = 'include thermal conduction')
    
    gen.controls(t_start = 0, t_end = 2.e-8, restart = True)

    gen.parameters(time_between_snapshots = 1e-9, initial_timestep = 1.e-14)
    
    return gen





In [8]:
import generator_object, write_run_plot, serial_sim_tools, new_plt_file
import numpy as np

from importlib import reload
for obj in [generator_object, write_run_plot, serial_sim_tools, new_plt_file]:
    reload(obj)
def sim3(var2):

    gen = generator_object.User_input()
    n_atom = 50
    rho, T_ev = 15, var2
    z = 12
    N0, N1 = 1, 501 #We are operating in 1d wih 40 nodes
    Rmin, Rmax = 0, 0.01
    p2 = 1.e-9


    gen.materials_atom(index = 1, quantum_n_max = 3, element= "sn")  
    gen.materials_region(nodes = [N0, N1], elec_temp = T_ev, qstart = True) 

    gen.materials_region_material(rho, n_atom, z, z**2) #

    gen.geometry(type = 'slab')
    gen.geometry_nodes(coordinate = "r", scaling_type  = "geom", nodes = [N0, N1], nodes_range = [Rmin, Rmax], drmin = 1.e-6, slope = -1)


    gen.sources_laser(index = 1, laser_wavelength = 1, option_1= 'value', option_2= 'history', id_value = 1, multiplier=1,)
    gen.sources_lasray(entrance_position =  10, entrance_direction_mu = 1, entrance_direction_phi = 0, fractional_power= 1, res_frac=.5)
    
    gen.sources_history(id = 1, value_multiplier = 1e18, time_multiplier = 1., pulse_type='gaussian', p1 = 5*p2, p2 = p2)

    gen.popular_switches(timestep_between_snapshot = 500, 
                        temparture_calc_heating_rates = ['temp calc = time dependant', 'heating rates = electronic'], 
                        raytrace = True, 
                        include_degeneracy = 'no degeneracy',
                        continuum_transfer_evolves_temp = True)

        
    gen.other_switches(resonant_absorption_fraction = 'constant value for each ray from lasray', 
                        subcycle_maximum = 1000,
                        do_kinetics_zone_centered = True, 
                        population_calculation = 'time dependent diffusion', 
                        control_calc_thermal_conduct = 'include thermal conduction')
    
    gen.controls(t_start = 0, t_end = 2.e-8, restart = True)

    gen.parameters(time_between_snapshots = 1e-9, initial_timestep = 1.e-14)

    gen.add_plot(name='electron temperature', xvar =  'time', yvar = 'te')
    gen.add_plot(name='Net ionization rate of isosequence', xvar =  'time', yvar = 'isoraten')
    return gen

name = 'johnrun3'
gen = sim3(2)
gen.plots
#write_run_plot.all(name=name, longprint = False, plot_duplicates=False, object=gen)
#write_run_plot.write(name=name, longprint = False, plot_duplicates=False, object=gen)
#write_run_plot.run(name=name, longprint = False, plot_duplicates=False, object=gen)
#write_run_plot.plot(name=name, longprint = False, plot_duplicates=False, object=gen)
write_run_plot.extra_plot(name = name)

{'electron temperature':               time           te
0                $          NaN
1    0.0000000E+00  23209.03900
2    1.0000000E-14  23209.03900
3    2.5000000E-14  23209.03900
4    4.7500000E-14  23209.03900
..             ...          ...
200  1.1795658E-08   2236.31390
201  1.2942502E-08   1772.40460
202  1.4662769E-08   1408.45450
203  1.7243169E-08   1120.54570
204  2.1113768E-08    891.23006

[205 rows x 2 columns], 'Net ionization rate of isosequence':               time  isoraten
0                $       NaN
1    0.0000000E+00       0.0
2    1.0000000E-14       0.0
3    2.5000000E-14       0.0
4    4.7500000E-14       0.0
..             ...       ...
200  1.1795658E-08       0.0
201  1.2942502E-08       0.0
202  1.4662769E-08       0.0
203  1.7243169E-08       0.0
204  2.1113768E-08       0.0

[205 rows x 2 columns]}


In [14]:
var = np.array([.5, .75, 1, 1.33, 2.])
Tvals = 592./11600*var

sims = []

for entry in Tvals:
    entry = round(entry, 5)
    gen = sim2(entry)
    base_name = 'johnrun2'
    name = f'{base_name}_{entry}'
    sims.append(name)
    write_run_plot.all(name=name, longprint = False, plot_duplicates=False, object=gen)
serial_sim_tools.plot_all(f'{base_name}_multiplot', sims)


writing to /home/brewster/Desktop/cretin_package-master/Personal_experiments/johnrun2_0.02552/johnrun2_0.02552.gen
running cretin with johnrun2_0.02552

writing to /home/brewster/Desktop/cretin_package-master/Personal_experiments/johnrun2_0.03828/johnrun2_0.03828.gen
running cretin with johnrun2_0.03828

writing to /home/brewster/Desktop/cretin_package-master/Personal_experiments/johnrun2_0.05103/johnrun2_0.05103.gen
running cretin with johnrun2_0.05103

writing to /home/brewster/Desktop/cretin_package-master/Personal_experiments/johnrun2_0.06788/johnrun2_0.06788.gen
running cretin with johnrun2_0.06788

writing to /home/brewster/Desktop/cretin_package-master/Personal_experiments/johnrun2_0.10207/johnrun2_0.10207.gen
running cretin with johnrun2_0.10207
multiplot to /home/brewster/Desktop/cretin_package-master/Personal_experiments/sims_multiplot
